A mixture of $N =10$ random Gaussians centered in $[-1,1]^n$ 
$$
f_{\mu,\sigma}(x, y) = \sum_{i=1}^{N} \frac{1}{\sigma_i\sqrt{2\pi}}\exp\left(-\frac{(x-\mu_i)^2}{2\sigma_i^2}\right)
$$
 is defined over the square $[-1, 1]^2$.

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/Globtim.jl`


In [2]:
# Constants and Parameters
const n, a, b = 2, 1, 1
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 1.5e-3             # Define the tolerance for the L2-norm

N = 12
params = init_gaussian_params(n, N, 0.2)
# Create a closure that captures params
rand_gaussian_closure = (x) -> rand_gaussian(x, params)
f = rand_gaussian_closure; # Function to be optimized

In [3]:
d = 14 # Initial Degree 
SMPL = 40 # Number of samples
center = [0.0, 0.0]
TR = test_input(n, center, (alpha, delta), tol_l2, (0.0, 0.0), 1.0, 1.0, f)
pol_cheb = Constructor(TR, d, GN=SMPL, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=SMPL, basis=:legendre);

current L2-norm: 0.0021594499966415184
Number of samples: 40
current L2-norm: 0.0027831608107755077
Number of samples: 40


We need to carefully assert the type of the coefficients of the polynomial approximant. 

In [4]:
@polyvar(x[1:n]) # Define polynomial ring 

(Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}[x₁, x₂],)

In [5]:
real_pts_cheb = solve_polynomial_system(x, n, d, pol_cheb.coeffs; basis=:chebyshev, bigint=true)

Tracking 169 paths...   1%|▍                            |  ETA: 0:17:48



Tracking 169 paths... 100%|█████████████████████████████| Time: 0:00:13
  # paths tracked:                  169
  # non-singular solutions (real):  169 (73)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         169 (73)


73-element Vector{Vector{Float64}}:
 [0.9430843951063741, 0.964577372338349]
 [0.8062999462354902, 0.842842704773808]
 [0.630014016733347, 0.9935644067324458]
 [0.2107114245598818, 0.9836686442236442]
 [-0.2434709797987138, 0.9803553078729993]
 [-0.8728039431279442, 0.9741947595121803]
 [-0.5206346080521944, 0.9774789665627016]
 [-0.02848037335711457, 0.9541856084098406]
 [0.5094164820361855, 0.9371613615386203]
 [0.8307081347805755, 0.9828137217783487]
 ⋮
 [-0.9107368163774793, 0.47156792167373046]
 [-0.95516674824385, -0.4116076761568695]
 [0.5934824741833704, 0.29847645308248644]
 [0.9625750058239845, 0.7662369196793417]
 [-0.9724524828678454, 0.6684959585279183]
 [-0.6180412995247679, 0.7923486411962857]
 [-0.9362089826106482, 0.8243495642003709]
 [-0.35275075451497934, 0.6829891809443008]
 [0.5926136178534671, 0.8138764784280104]

In [6]:
real_pts_lege = solve_polynomial_system(x, n, d, pol_lege.coeffs; basis=:legendre, bigint=true)

Tracking 169 paths...   1%|▍                            |  ETA: 0:08:17



Tracking 169 paths...   2%|▋                            |  ETA: 0:04:25



Tracking 169 paths...   5%|█▍                           |  ETA: 0:02:16



Tracking 169 paths...   7%|██                           |  ETA: 0:01:33



Tracking 169 paths...   9%|██▋                          |  ETA: 0:01:15



Tracking 169 paths...  11%|███▏                         |  ETA: 0:01:04



Tracking 169 paths...  13%|███▊                         |  ETA: 0:00:52



Tracking 169 paths...  15%|████▎                        |  ETA: 0:00:46



Tracking 169 paths...  17%|████▊                        |  ETA: 0:00:42



Tracking 169 paths...  18%|█████▍                       |  ETA: 0:00:38



Tracking 169 paths...  21%|██████                       |  ETA: 0:00:33



Tracking 169 paths...  22%|██████▍                      |  ETA: 0:00:32



Tracking 169 paths...  24%|██████▉                      |  ETA: 0:00:30



Tracking 169 paths...  25

77-element Vector{Vector{Float64}}:
 [0.9449856391394038, 0.9650190247505129]
 [0.8214265539095055, 0.978659008677818]
 [0.4872276279888685, 0.914554542682982]
 [-0.028323531431398638, 0.9667207165285364]
 [-0.5182103426549473, 0.9756700587410526]
 [-0.837723808989232, 0.9626097954072877]
 [-0.24378182426996103, 0.9807042035622986]
 [0.18720133885873994, 0.9796023169437212]
 [0.9673723734481704, 0.7470780441715374]
 [0.6047234650201172, 0.9814730484259109]
 ⋮
 [-0.3943413179132941, 0.5647265962981283]
 [-0.7458122807394646, 0.8898501636187603]
 [-0.34350997491787266, 0.8853530258660812]
 [0.982280847871723, 0.5486515873673584]
 [0.3828187148921163, 0.9858748034883416]
 [-0.841691081364353, 0.6958800563066627]
 [-0.972247522090725, 0.6551802462159982]
 [0.22237606823573589, 0.5659346660018278]
 [0.8099002677212249, 0.5697056740579476]

In [7]:
# Usage example:
df_cheb = process_critical_points(real_pts_cheb, f, scale_factor)
df_lege = process_critical_points(real_pts_lege, f, scale_factor)

Row,x1,x2,z
,Float64,Float64,Float64
1,0.944986,0.965019,-5.85641e-7
2,0.821427,0.978659,-1.51123e-5
3,0.487228,0.914555,-0.0208849
4,-0.0283235,0.966721,-0.00373004
5,-0.51821,0.97567,2.54305e-6
6,-0.837724,0.96261,2.49785e-7
7,-0.243782,0.980704,-4.78287e-5
8,0.187201,0.979602,-0.0189169
9,0.967372,0.747078,-0.00245378


In [8]:
using GLMakie

# Extract coordinates and function values
# Change the coordinates to uniform grid
coords = pol_cheb.scale_factor * pol_lege.grid
z_coords = pol_lege.z

if size(coords)[2] == 2  # Plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Sample Points",
        xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Define threshold for switching between scatter and surface
    point_threshold = 1000  # Adjust this value based on your needs
    sample_fraction = 0.2   # Fraction of points to use if exceeding threshold

    if length(z_coords) > point_threshold
        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Determine grid dimensions
        nx = length(x_unique)
        ny = length(y_unique)

        # Reshape data into a grid
        z_grid = reshape(z_coords, nx, ny)

        # Create surface plot
        surface!(ax, x_unique, y_unique, z_grid,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Sample a fraction of points for scatter overlay
        n_points = length(z_coords)
        sample_indices = rand(1:n_points, Int(floor(n_points * sample_fraction)))

        # Plot sampled scatter points
        scatter!(ax, coords[sample_indices, 1],
            coords[sample_indices, 2],
            z_coords[sample_indices],
            markersize=2,
            color=:black,
            label="Sampled Data Points")
    else
        # Original scatter plot for smaller datasets
        scatter!(ax, coords[:, 1], coords[:, 2], z_coords,
            markersize=2, color=:black,
            label="Sampled Data Core")

        scatter!(ax, coords[:, 1], coords[:, 2], z_coords,
            markersize=4, color=z_coords,
            colormap=:viridis,
            label="Sampled Data Halo")
    end

    # Plot the critical points
    scatter!(ax, df_cheb.x1, df_cheb.x2, df_cheb.z,
        markersize=10, color=:orange,
        label="Chebyshev approximant critical points")

    scatter!(ax, df_lege.x1, df_lege.x2, df_lege.z,
        markersize=10, color=:yellow,
        label="Legendre approximant critical points")

    display(fig)
end

GLMakie.Screen(...)

In [ ]:
# GLMakie.closeall()